In [2]:
import pandas as pd


In [3]:
# Loading the raw data
df_pay = pd.read_csv('Payments.csv')
df_cli = pd.read_csv('Clients.csv')


In [4]:
# converting epoch date
df_pay["transaction_date"] = pd.to_datetime(df_pay["transaction_date"], unit="s")

In [28]:
# skimming the data
print(df_pay.head(), df_pay.info(), df_pay.describe())
print(df_cli.head(), df_cli.info(), df_cli.describe())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25559 entries, 0 to 25558
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    25559 non-null  int64  
 1   contract_id       25559 non-null  int64  
 2   client_id         25559 non-null  int64  
 3   transaction_date  25559 non-null  int64  
 4   payment_amt       25559 non-null  float64
 5   payment_code      25559 non-null  object 
dtypes: float64(1), int64(4), object(1)
memory usage: 1.2+ MB
   transaction_id  contract_id  client_id  transaction_date  payment_amt  \
0           20175          927          1        1527012511        66.66   
1            8485          927          1        1511716095        66.66   
2           13778          927          1        1519319303        66.66   
3           22768          927          1        1529863724        66.66   
4           15698          927          1        1521738504        66.66  

In [29]:
# checking all duplicates and missing keys 
print('Duplicate transactions:',
      df_pay['transaction_id'].duplicated().sum())

print('Missing client IDs in payments:',
      df_pay['client_id'].isna().sum(),
      'unique missing contracts:',
      df_pay.loc[df_pay['client_id'].isna(), 'contract_id'].nunique())


Duplicate transactions: 0
Missing client IDs in payments: 0 unique missing contracts: 0


In [5]:
#cleaning the data, trimming and uppercase
df_pay['payment_code'] = (
    df_pay['payment_code']
      .astype(str)
      .str.strip()
      .str.upper()
)

In [6]:
df_pay['payment_amt'] = pd.to_numeric(df_pay['payment_amt'], errors='coerce')
# dropping negative or missing amounts
df_pay = df_pay[df_pay['payment_amt'] > 0]


In [7]:

df_pay['IsDefault'] = df_pay['payment_code'] == 'DEFAULT'


In [33]:
#cleaning data, trimming and capslock
df_cli['entity_type'] = (
    df_cli['entity_type']
      .astype(str)
      .str.strip()
      .str.title() 
)


In [34]:
#calculating business age
df_cli['entity_year_established'] = pd.to_numeric(
    df_cli['entity_year_established'], errors='coerce'
)
current_year = pd.Timestamp.now().year
df_cli['BusinessAge'] = current_year - df_cli['entity_year_established']


In [35]:
# Counting how many times each client_id appears
dup_counts = df_cli['client_id'].value_counts()

# checking duplicates with more than 1
dup_clients = dup_counts[ dup_counts > 1 ]
print("Clients with duplicate IDs:\n", dup_clients)


Clients with duplicate IDs:
 client_id
591     3
165     2
473     2
1262    2
797     2
Name: count, dtype: int64


In [36]:
 # first duplicate client_id
example = dup_clients.index[0]  
print(df_cli[df_cli['client_id'] == example])


     client_id                 entity_type  entity_year_established  \
245        591  Australian Private Company                     2013   
306        591  Australian Private Company                     2007   
816        591  Australian Private Company                     2015   

     BusinessAge  
245           12  
306           18  
816           10  


In [18]:
df_cli_unique = df_cli.drop_duplicates(subset='client_id', keep='first')


In [38]:
df_merge = df_pay.merge(
    df_cli_unique,
    on='client_id',
    how='left',
    validate='many_to_one'   
)


In [37]:
# Finding overall default rate
default_rate = df_pay['IsDefault'].mean()
print(f'Overall default rate: {default_rate:.1%}')

Overall default rate: 8.7%


In [26]:
df_merge.to_csv('merged_payments_clients.csv', index=False)